<a href="https://colab.research.google.com/github/ivyyang94134/GPT-SoVITS/blob/main/FB%E7%A4%BE%E7%BE%A4%E7%88%AC%E8%9F%B2%E6%95%99%E5%AD%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##FB 貼文爬蟲

##下載 寶可夢多張圖片

In [1]:
!pip install requests beautifulsoup4 gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.3 MB/s

In [2]:
import requests
from bs4 import BeautifulSoup
import os
import concurrent.futures
import gradio as gr

def download_pokemon_image(pokemon_id):
    url = f'https://tw.portal-pokemon.com/play/pokedex/{pokemon_id:04d}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    img = soup.select('meta[property="og:image"]')
    if img:
        img_url = img[0]['content']
        img_response = requests.get(img_url)
        if img_response.status_code == 200:
            filename = f'pokemon/{pokemon_id:04d}.png'
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, 'wb') as f:
                f.write(img_response.content)
            return filename
    return None

def download_multiple_pokemon(start_id, end_id):
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(download_pokemon_image, i): i for i in range(start_id, end_id+1)}
        downloaded_files = []
        for future in concurrent.futures.as_completed(future_to_id):
            filename = future.result()
            if filename:
                downloaded_files.append(filename)
    return downloaded_files

def download_and_display(start_id, end_id):
    files = download_multiple_pokemon(start_id, end_id)
    return files

iface = gr.Interface(
    fn=download_and_display,
    inputs=[
        gr.Number(label="起始ID", minimum=1, maximum=1000, step=1),
        gr.Number(label="結束ID", minimum=1, maximum=1000, step=1)
    ],
    outputs=gr.Gallery(label="下載的寶可夢圖片"),
    title="寶可夢圖片下載器",
    description="輸入起始和結束的寶可夢ID,下載並顯示對應的寶可夢圖片。"
)

if __name__ == "__main__":
    iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4cc4be8dc821ad8e4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##PTT扒光正妹圖片
~~~
https://www.ptt.cc/bbs/Beauty/index.html

In [ ]:
!pip install --upgrade gradio

In [ ]:
import requests
from bs4 import BeautifulSoup
import tempfile
import gradio as gr
from google.colab import output
import urllib3
import json

output.enable_custom_widget_manager()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def fetch_ptt_images(url):
    try:
        print(f"Attempting to fetch images from URL: {url}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        web = requests.get(url, cookies={'over18':'1'}, headers=headers, verify=False, allow_redirects=False)

        print(f"Request status code: {web.status_code}")
        print(f"Response headers: {json.dumps(dict(web.headers), indent=2)}")
        print(f"Response content (first 500 characters): {web.text[:500]}")

        if web.status_code == 302:
            print(f"Redirect detected. Location: {web.headers.get('Location')}")
            return []

        web.raise_for_status()

        soup = BeautifulSoup(web.text, "html.parser")
        main_content = soup.find('div', id='main-content')
        if main_content:
            imgs = main_content.find_all('img')
            print("Found main-content div")
        else:
            imgs = soup.find_all('img')
            print("Main-content div not found, searching in entire page")

        image_urls = [img['src'] for img in imgs]
        print(f"Found {len(image_urls)} images")
        print(f"Image URLs: {image_urls}")
        return image_urls
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []
    except Exception as e:
        print(f"Error fetching images: {e}")
        return []

def display_images(url):
    image_urls = fetch_ptt_images(url)
    if not image_urls:
        print("No images found or error occurred")
        return []
    return image_urls  # 直接返回圖片 URL，而不是下載圖片

def on_select(evt: gr.SelectData, images):
    return images[evt.index]

def fetch_and_display(url):
    images = display_images(url)
    error = "" if images else "無法獲取圖片,請檢查 URL 是否正確。詳細錯誤信息請查看控制台輸出。"
    return images, error

with gr.Blocks() as demo:
    gr.Markdown("# PTT 正妹下載器")
    with gr.Row():
        url_input = gr.Textbox(label="輸入 PTT 網頁 URL")
        fetch_button = gr.Button("獲取圖片")

    gallery = gr.Gallery(label="圖片預覽", columns=4)
    error_output = gr.Textbox(label="錯誤信息")

    fetch_button.click(fetch_and_display, inputs=url_input, outputs=[gallery, error_output])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://566d2285833f5bc8ee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Attempting to fetch images from URL: https://www.ptt.cc/bbs/Beauty/M.1721122504.A.E63.html
Request status code: 200
Response headers: {
  "Date": "Tue, 16 Jul 2024 12:16:35 GMT",
  "Content-Type": "text/html; charset=utf-8",
  "Transfer-Encoding": "chunked",
  "Connection": "keep-alive",
  "strict-transport-security": "max-age=0",
  "CF-Cache-Status": "DYNAMIC",
  "Report-To": "{\"endpoints\":[{\"url\":\"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=NT9IeKkldxTRmBzIOz%2Bz7mV1WbTiYz%2BpwsJPVl77c3WA2RHpDU3qGIrBdcUcpsLjm8pMjpEkYWOjNqotHxLykac9lfZk9CH797pe8%2FLpJ2WYOORNElsnSYD73NKX\"}],\"group\":\"cf-nel\",\"max_age\":604800}",
  "NEL": "{\"success_fraction\":0,\"report_to\":\"cf-nel\",\"max_age\":604800}",
  "X-Content-Type-Options": "nosniff",
  "Server": "cloudflare",
  "CF-RAY": "8a41ec9e0ecdab97-SJC-PIG",
  "Content-Encoding": "gzip",
  "alt-svc": "h3=\":443\"; ma=86400"
}
Response content (first 500 characters): <!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta nam

##以GAS 做爬蟲
~~~
// 主函數：爬取 Facebook 頁面
function scrapeFacebookPage() {
  var ui = SpreadsheetApp.getUi();
  
  // 提示用戶手動登入
  var loginResult = ui.alert(
    '手動登入提示',
    '請在瀏覽器中手動登入 Facebook，然後點擊"確定"繼續。',
    ui.ButtonSet.OK_CANCEL
  );
  
  if (loginResult != ui.Button.OK) {
    Logger.log('用戶取消了操作');
    return;
  }
  
  // 獲取用戶的 cookies
  var cookies = getCookiesFromUser();
  if (!cookies) {
    Logger.log('未能獲取有效的 cookies，無法繼續爬取');
    return;
  }

  // 設置請求選項
  var options = {
    'method': 'get',
    'headers': {
      'Cookie': cookies
    },
    'muteHttpExceptions': true // 不拋出 HTTP 錯誤，以便我們可以自行處理
  };

  // 準備 Google 試算表
  var sheet = SpreadsheetApp.getActiveSpreadsheet().getActiveSheet();
  sheet.clear(); // 清空試算表
  // 添加欄位標題
  sheet.appendRow(['發布日期', '發布時間', '作者', '貼文內容', '讚數', '留言數', '分享數', '貼文連結']);

  var postsScraped = 0; // 已爬取的貼文數量
  var nextPageUrl = CONFIG.targetPage; // 初始頁面 URL

  // 主要爬取循環
  while (postsScraped < CONFIG.maxPosts && nextPageUrl) {
    // 添加延遲以避免被封鎖
    Utilities.sleep(CONFIG.delayBetweenRequests);

    try {
      // 獲取頁面內容
      var pageResponse = UrlFetchApp.fetch(nextPageUrl, options);
      var pageContent = pageResponse.getContentText();

      Logger.log('頁面響應狀態: ' + pageResponse.getResponseCode());
      Logger.log('頁面內容片段: ' + pageContent.substring(0, 500) + '...'); // 記錄頁面內容的前500個字符

      // 提取貼文
      var posts = extractPosts(pageContent);

      if (posts.length > 0) {
        Logger.log('找到 ' + posts.length + ' 條貼文');
        for (var i = 0; i < posts.length && postsScraped < CONFIG.maxPosts; i++) {
          var post = posts[i];
          // 將提取的數據添加到試算表中
          sheet.appendRow([
            post.date, post.time, post.author, post.content,
            post.likes, post.comments, post.shares, post.link
          ]);
          postsScraped++;
        }
      } else {
        Logger.log('在頁面中未找到貼文');
        Logger.log('頁面標題: ' + extractPageTitle(pageContent));
      }

      // 獲取下一頁的 URL
      nextPageUrl = extractNextPageUrl(pageContent);
    } catch (error) {
      Logger.log('訪問頁面時發生錯誤: ' + error.message);
      throw error; // 重新拋出錯誤以停止執行
    }
  }

  Logger.log('爬蟲完成，共爬取了 ' + postsScraped + ' 條貼文。');
}

// 從用戶獲取 cookies
function getCookiesFromUser() {
  var ui = SpreadsheetApp.getUi();
  var result = ui.prompt(
    '輸入 Facebook Cookies',
    '請從瀏覽器中複製 Facebook 的 cookies 並粘貼在這裡：',
    ui.ButtonSet.OK_CANCEL
  );

  if (result.getSelectedButton() == ui.Button.OK) {
    return result.getResponseText();
  } else {
    return null;
  }
}

// 提取貼文（更靈活的方法）
function extractPosts(pageContent) {
  var posts = [];
  var postRegex = /<div[^>]*?role="article"[^>]*>([\s\S]*?)<\/div>\s*<div[^>]*?role="article"/g;
  var match;

  while ((match = postRegex.exec(pageContent)) !== null) {
    var postContent = match[1];
    posts.push({
      date: extractDate(postContent),
      time: extractTime(postContent),
      author: extractAuthor(postContent),
      content: extractContent(postContent),
      likes: extractReactions(postContent, '讚'),
      comments: extractReactions(postContent, '留言'),
      shares: extractReactions(postContent, '分享'),
      link: extractPostLink(postContent)
    });
  }

  return posts;
}

// 提取日期
function extractDate(postContent) {
  var match = postContent.match(/(\d{1,2})月(\d{1,2})日/);
  return match ? match[0] : '未知日期';
}

// 提取時間
function extractTime(postContent) {
  var match = postContent.match(/(\d{1,2}):(\d{2})/);
  return match ? match[0] : '未知時間';
}

// 提取作者
function extractAuthor(postContent) {
  var match = postContent.match(/<h3[^>]*><a[^>]*>(.*?)<\/a>/);
  return match ? match[1] : '未知作者';
}

// 提取貼文內容
function extractContent(postContent) {
  var match = postContent.match(/<p>([\s\S]*?)<\/p>/);
  return match ? match[1].trim() : '無內容';
}

// 提取反應數（讚、留言、分享）
function extractReactions(postContent, type) {
  var match = postContent.match(new RegExp(type + '\\s*(\\d+)'));
  return match ? match[1] : '0';
}

// 提取貼文連結
function extractPostLink(postContent) {
  var match = postContent.match(/href="(\/(?:groups|[^\/]+)\/[^"]*?)"/);
  return match ? 'https://mbasic.facebook.com' + match[1] : '無連結';
}

// 提取下一頁 URL
function extractNextPageUrl(pageContent) {
  var match = pageContent.match(/href="([^"]*?)">顯示更多<\/a>/);
  return match ? 'https://mbasic.facebook.com' + match[1] : null;
}

// 提取頁面標題
function extractPageTitle(pageContent) {
  var match = pageContent.match(/<title>(.*?)<\/title>/);
  return match ? match[1] : '無標題';
}

// 創建一個選單項來運行爬蟲
function onOpen() {
  var ui = SpreadsheetApp.getUi();
  ui.createMenu('Facebook 爬蟲')
    .addItem('爬取 Facebook 頁面', 'scrapeFacebookPage')
    .addItem('更改爬取目標', 'changeTargetPage')
    .addToUi();
}

// 用於更改爬取目標的函數
function changeTargetPage() {
  var ui = SpreadsheetApp.getUi();
  var result = ui.prompt(
    '更改爬取目標',
    '請輸入要爬取的 Facebook 頁面 mbasic URL：',
    ui.ButtonSet.OK_CANCEL);

  var button = result.getSelectedButton();
  var text = result.getResponseText();
  if (button == ui.Button.OK) {
    CONFIG.targetPage = text;
    ui.alert('爬取目標已更改為：' + text);
  } else if (button == ui.Button.CANCEL) {
    ui.alert('已取消更改。');
  } else if (button == ui.Button.CLOSE) {
    ui.alert('您關閉了對話框。');
  }
}

// 注意事項：
// 1. 此腳本需要用戶手動登入 Facebook 並提供 cookies，以避免自動登入可能帶來的帳戶風險。
// 2. 使用爬蟲可能違反 Facebook 的服務條款。請確保您有權限訪問和使用您爬取的數據。
// 3. Facebook 可能會改變其網頁結構，導致爬蟲失效。您可能需要定期更新正則表達式以匹配新的 HTML 結構。
// 4. 本腳本僅供教育和研究目的使用。在實際應用中，請遵守所有相關法律和平台政策。

~~~

In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
# 導入必要的庫
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

# 設置配置信息
CONFIG = {
    'targetPage': 'https://mbasic.facebook.com/groups/3208727879422567/',  # 目標 Facebook 頁面的 mbasic URL
    'maxPosts': 50,  # 最多爬取的貼文數量
    'delayBetweenRequests': 5  # 每次請求之間的延遲（秒）
}

def get_cookies_from_user():
    """
    從用戶獲取 Facebook cookies
    返回：包含 cookies 的字典
    """
    cookies_string = input("請從瀏覽器中複製 Facebook 的 cookies 並粘貼在這裡：")
    cookies = {}
    for cookie in cookies_string.split(';'):
        key, value = cookie.strip().split('=', 1)
        cookies[key] = value
    return cookies

def scrape_facebook_page(cookies):
    """
    爬取 Facebook 頁面
    參數：cookies - 包含用戶 cookies 的字典
    返回：爬取到的貼文列表
    """
    posts = []
    next_page_url = CONFIG['targetPage']

    while len(posts) < CONFIG['maxPosts'] and next_page_url:
        print(f"正在爬取頁面：{next_page_url}")
        response = requests.get(next_page_url, cookies=cookies)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 提取當前頁面的所有貼文
        articles = soup.find_all('div', role='article')
        for article in articles:
            if len(posts) >= CONFIG['maxPosts']:
                break
            post = extract_post_info(article)
            posts.append(post)

        # 尋找下一頁的連結
        next_page_link = soup.find('a', string='顯示更多')
        next_page_url = 'https://mbasic.facebook.com' + next_page_link['href'] if next_page_link else None

        # 延遲以避免被封鎖
        time.sleep(CONFIG['delayBetweenRequests'])

    return posts

def extract_post_info(article):
    """
    從 HTML 中提取單個貼文的信息
    參數：article - 包含貼文信息的 BeautifulSoup 對象
    返回：包含貼文信息的字典
    """
    # 提取作者
    author_tag = article.find('h3')
    author = author_tag.text if author_tag else "未知作者"

    # 提取發布時間
    time_tag = article.find('abbr')
    post_time = time_tag.text if time_tag else "未知時間"

    # 提取貼文內容
    content_tag = article.find('p')
    content = content_tag.text if content_tag else "無內容"

    # 提取互動數據（讚、留言、分享）
    footer = article.find('footer')
    likes = comments = shares = 0
    if footer:
        likes_match = re.search(r'(\d+)\s*讚', footer.text)
        comments_match = re.search(r'(\d+)\s*則留言', footer.text)
        shares_match = re.search(r'(\d+)\s*次分享', footer.text)
        likes = int(likes_match.group(1)) if likes_match else 0
        comments = int(comments_match.group(1)) if comments_match else 0
        shares = int(shares_match.group(1)) if shares_match else 0

    # 提取貼文連結
    link_tag = article.find('a', string='完整動態')
    link = 'https://mbasic.facebook.com' + link_tag['href'] if link_tag else "無連結"

    return {
        '作者': author,
        '發布時間': post_time,
        '內容': content,
        '讚數': likes,
        '留言數': comments,
        '分享數': shares,
        '連結': link
    }

# 主程序
if __name__ == "__main__":
    print("Facebook 爬蟲程式啟動")
    print("請確保您已經手動登入 Facebook")
    cookies = get_cookies_from_user()
    posts = scrape_facebook_page(cookies)

    # 將爬取的數據轉換為 DataFrame
    df = pd.DataFrame(posts)

    # 顯示爬取結果的摘要
    print(f"\n共爬取了 {len(posts)} 條貼文")
    print(df.head())

    # 將結果保存為 CSV 檔案
    df.to_csv('facebook_posts.csv', index=False)
    print("\n爬取的數據已保存到 facebook_posts.csv 檔案中")

# 注意事項：
# 1. 此腳本需要用戶手動登入 Facebook 並提供 cookies，以避免自動登入可能帶來的帳戶風險。
# 2. 使用爬蟲可能違反 Facebook 的服務條款。請確保您有權限訪問和使用您爬取的數據。
# 3. Facebook 可能會改變其網頁結構，導致爬蟲失效。您可能需要定期更新提取邏輯以匹配新的 HTML 結構。
# 4. 本腳本僅供教育和研究目的使用。在實際應用中，請遵守所有相關法律和平台政策。

Facebook 爬蟲程式啟動
請確保您已經手動登入 Facebook
請從瀏覽器中複製 Facebook 的 cookies 並粘貼在這裡：# Netscape HTTP Cookie File # http://curl.haxx.se/rfc/cookie_spec.html # This is a generated file!  Do not edit.  .facebook.com	TRUE	/	TRUE	1755685710	sb	1OIaZcN75qrjAKYRc3tyV3jJ .facebook.com	TRUE	/	TRUE	1730820821	datr	1OIaZQPOHb4jOL3ocPAvtft8 .facebook.com	TRUE	/	TRUE	1721735335	dpr	1.125 .facebook.com	TRUE	/	TRUE	1755685692	ps_n	1 .facebook.com	TRUE	/	TRUE	1755685692	ps_l	1 .facebook.com	TRUE	/	TRUE	1752661710	c_user	61561847643757 .facebook.com	TRUE	/	TRUE	1752661710	xs	38%3AE1RyxK2WSxmtKg%3A2%3A1721125711%3A-1%3A-1 .facebook.com	TRUE	/	TRUE	1728901710	fr	1DVhp2YqWYkRjoeC0.AWUjeEJvhs6HRZTGwhvpYGm96o8.BmlksL..AAA.0.0.BmlktO.AWX5x2wpq5A .facebook.com	TRUE	/	TRUE	1721735335	wd	904x772 .facebook.com	TRUE	/	TRUE	0	presence	C%7B%22t3%22%3A%5B%5D%2C%22utc3%22%3A1721130534003%2C%22v%22%3A1%7D .www.facebook.com	TRUE	/	TRUE	1755690535	m_ls	%7B%2261561847643757%22%3A%7B%22c%22%3A%7B%221%22%3A%22HCwAABYQFoKF36gFEwUW2vn7wK_

ValueError: not enough values to unpack (expected 2, got 1)

##進化套件 Facebook-Scraper

In [ ]:
# 安裝必要的套件
!pip install facebook-scraper pandas tqdm

In [ ]:
# 導入所需的庫
import pandas as pd
from facebook_scraper import get_posts
from datetime import datetime
from time import sleep
from random import randint
from tqdm import tqdm
import warnings

# 設置配置
CONFIG = {
    'fanpage': 'taipei33',  # 目標粉絲頁的 ID
    'pages': 200,  # 預設爬取頁數
    'year_limit': 2019,  # 設置年份限制，早於此年份的貼文不爬取
    'sleep_min': 10,  # 每次爬取後的最小睡眠時間（秒）
    'sleep_max': 60,  # 每次爬取後的最大睡眠時間（秒）
}

def get_cookies():
    """從使用者獲取 cookies"""
    print("請提供 Facebook cookies:")
    print("1. 登入 Facebook")
    print("2. 打開瀏覽器開發者工具（F12）")
    print("3. 在 Network 標籤中找到對 facebook.com 的請求")
    print("4. 在 Headers 中複製 Cookie 的值")
    return input("請將 cookies 粘貼在這裡: ").strip()

def scrape_facebook(cookies):
    """主要的爬蟲函數"""
    P = pd.DataFrame()
    i = 0

    print(f"開始爬取 {CONFIG['fanpage']} 的貼文...")

    try:
        for post in tqdm(get_posts(CONFIG['fanpage'], pages=CONFIG['pages'], cookies=cookies, options={"reactors": True}), desc="爬取進度"):
            if int(post['time'].strftime("%Y")) <= CONFIG['year_limit']:
                print(f"已達到年份限制 {CONFIG['year_limit']}，停止爬取")
                break

            P = P.append({
                'user_id': str(post.get('user_id', '')),
                'username': str(post.get('username', '')),
                'time': post['time'],
                'post_url': post['post_url'],
                'post_id': str(post['post_id']),
                'post_text': post.get('post_text', '').strip().replace("\n", " "),
                'like_count': post['reactions'].get('讚', 0) if post.get('reactions') else 0,
                'love_count': post['reactions'].get('大心', 0) if post.get('reactions') else 0,
                'go_count': post['reactions'].get('加油', 0) if post.get('reactions') else 0,
                'wow_count': post['reactions'].get('哇', 0) if post.get('reactions') else 0,
                'haha_count': post['reactions'].get('哈', 0) if post.get('reactions') else 0,
                'sad_count': post['reactions'].get('嗚', 0) if post.get('reactions') else 0,
                'angry_count': post['reactions'].get('怒', 0) if post.get('reactions') else 0,
                'share_count': post.get('shares', 0),
                'comment_count': post.get('comments', 0),
            }, ignore_index=True)

            i += 1
            print(f"\n爬取完成 {i}. POST_ID: {post['post_id']} {post['time']}\nURL: {post['post_url']}\n")

            sleep_time = randint(CONFIG['sleep_min'], CONFIG['sleep_max'])
            print(f"休息 {sleep_time} 秒...")
            sleep(sleep_time)

    except Exception as e:
        print(f"爬取過程中發生錯誤: {str(e)}")

    return P

def main():
    warnings.filterwarnings("ignore")
    cookies = get_cookies()

    df = scrape_facebook(cookies)

    if not df.empty:
        filename = f"{CONFIG['fanpage']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"資料已保存到 {filename}")
        print(f"共爬取了 {len(df)} 條貼文")
    else:
        print("沒有爬取到任何資料")

if __name__ == "__main__":
    main()

# 注意事項：
# 1. 此腳本需要使用者提供 Facebook cookies，請確保安全使用且不要分享您的 cookies。
# 2. 使用爬蟲可能違反 Facebook 的服務條款。請確保您有權限訪問和使用您爬取的資料。
# 3. 設置適當的爬取間隔以避免被 Facebook 封鎖。
# 4. 本腳本僅供教育和研究目的使用。在實際應用中，請遵守所有相關法律和平台政策。